CHargement du df_machine_learning depuis le drive partagé

In [ ]:
from google.colab import drive
import pandas as pd
import os
from io import StringIO
# Monter Google Drive
drive.mount('/content/drive', force_remount= True) #force_remount = True permet d'avoir un fichier mis à jour si modifs sur google drive
df=pd.read_csv('/content/drive/MyDrive/Datascientest/Projet_accidents/Dataset/2019_2023/df_machine_learning.csv')
df.gravité_accident= df.gravité_accident.astype('category')
df.info()

SVM/Random Forest avec choix de variables


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier  # RandomForest à la place de LogisticRegression
from sklearn.svm import SVC  # Utilisation de SVM
from imblearn.over_sampling import SMOTE  # Over-sampling
from imblearn.pipeline import Pipeline as ImbPipeline # Pipeline compatible avec sampling
from sklearn.model_selection import GridSearchCV

# 🔹 Classe de transformation cyclique de l'heure
class CyclicalFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, period=24):
        self.period = period

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.astype(float)
        X_sin = np.sin(2 * np.pi * X / self.period)
        X_cos = np.cos(2 * np.pi * X / self.period)
        return np.c_[X_sin, X_cos]

# 🎯 Séparation des variables explicatives et cible
X = df.drop(['Num_Acc', 'lat', 'long', 'jour', 'mois', 'date', 'an', 'hrmn', 'dep', 'int', 'col', 'tué',
             'blessé_léger', 'blessé_hospitalisé', 'gravité_accident','nationale_departementale_communale', 'autoroute', 'autre_route',
       'sens_unique', 'bidirectionnel', 'route_seche',
       'route_mouillee_enneigee', 'etat_route_autre',
       'indemne', 'total_sans_secu', 'total_ceinture', 'total_casque',
       'total_secu_enfant', 'total_gilet', 'total_airbag', 'total_gants',
       'total_gants_airbag', 'total_autre', 'place_conducteur', 'pax_AV',
       'pax_AR', 'pax_Milieu', 'place_pieton','obstacle_fixe', 'obstacle_mobile', 'aucun_choc',
       'choc_AV', 'choc_AR', 'choc_cote', 'choc_tonneaux', 'VL_VU',
       '2roues_3roues_quad', 'PL', 'bus_car', 'velo_trott_edp', 'atm'], axis=1)
y = df['gravité_accident']

# 🎯 Séparation train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 🔹 Colonnes catégorielles et numériques
categorical_features = ['agg']
cyclical_features = ['heure']
passthrough_features = ['nbv', 'vma', 'usager_count', 'homme', 'femme', '0-17', '18-60', '61-95', 'nbr_veh']

# 🔹 Préprocessing
preprocessor = ColumnTransformer([
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
    ('cyclical', CyclicalFeatures(), cyclical_features)
], remainder='passthrough')

# 🔹 Pipeline avec SMOTE + SVM ou RandomForest
pipeline = ImbPipeline([
    ('preprocessor', preprocessor),
    ('sampling', SMOTE(random_state=42)),  # Over-sampling
    ('classifier', RandomForestClassifier(random_state=42))  # Remplacer par RandomForest
])

# Grille de recherche : plusieurs modèles et hyperparamètres
param_grid = [
    {  # SVM
        'classifier': [SVC()],
        'classifier__C': [0.1, 1, 10],  # Paramètre de régularisation
        'classifier__kernel': ['linear', 'rbf'],  # Types de noyaux à essayer
        'classifier__gamma': ['scale', 'auto']  # Paramètres du noyau RBF
    },
    {  # RandomForest
        'classifier': [RandomForestClassifier()],
        'classifier__n_estimators': [50, 100, 200],
        'classifier__max_depth': [None, 10, 20],
        'classifier__min_samples_split': [2, 5, 10],
        'classifier__min_samples_leaf': [1, 2, 4]
    }
]

# GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=2, scoring='f1_weighted', n_jobs=-1, verbose=2)

# Entraînement
grid_search.fit(X_train, y_train)

# Meilleurs paramètres trouvés
print(f"Meilleur modèle : {grid_search.best_estimator_.named_steps['classifier']}")
print(f"Meilleurs paramètres : {grid_search.best_params_}")

# Récupération du meilleur modèle
best_model = grid_search.best_estimator_

# 🔹 Évaluation avec Accuracy
from sklearn.metrics import classification_report, confusion_matrix

y_pred = best_model.predict(X_test)
print("\nClassification Report :")
print(classification_report(y_test, y_pred))

# Affichage de la matrice de confusion
plt.figure(figsize=(6, 6))
cm = confusion_matrix(y_test, y_pred)
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Matrice de confusion')
plt.colorbar()
plt.xticks(np.arange(len(np.unique(y))), np.unique(y), rotation=45)
plt.yticks(np.arange(len(np.unique(y))), np.unique(y))
plt.xlabel('Prédictions')
plt.ylabel('Vérité terrain')
plt.show()